# Verificamos si hay GPU disponible para que no tarde tanto en ejecutarse el entrenamiento

In [70]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando: {device}")
if device.type == 'cuda':
    print("GPU detectada. Ya puedes proceder a ejecutar el notebook")
else:
    print("Recomiendo activar modo GPU antes de avanzar")
    print("Para hacerlo: En Colab ve a Runtime > Change runtime type > GPU")

Usando: cuda
GPU detectada. Ya puedes proceder a ejecutar el notebook


# Descarga de imagenes para pruebas

In [71]:
import os

# Lista de imágenes a descargar
imagenes = ['sapo1.jpg', 'sapo2.jpg', 'sapo3.jpg']
base_url = 'https://raw.githubusercontent.com/JuanCruzMonteros/Vision-Computacional/main/tp4%20-%20pytorch/'

print("Verificando imágenes...")
for img in imagenes:
    if os.path.exists(img):
        print(f"✓ {img} ya existe, saltando descarga")
    else:
        print(f"↓ Descargando {img}...")
        !wget -q {base_url}{img}
        print(f"✓ {img} descargado")

Verificando imágenes...
✓ sapo1.jpg ya existe, saltando descarga
✓ sapo2.jpg ya existe, saltando descarga
✓ sapo3.jpg ya existe, saltando descarga


# IMPORTAR LIBRERÍAS

In [72]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2
from google.colab import files

# Preparamos el modelo, definimos y configuramos la red

In [73]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

## Agregamos optimizaciones para mejorar el entrenamiento:
* Como tenemos GPU aumento el batch a 128 en caso de tener GPU para procesar más imagenes por ciclo
* Por las pruebas, con 10 epocas es más que suficiente.

In [74]:
# Reducción del dataset
subset_size = 50000
subset_indices = list(range(subset_size))
subset = torch.utils.data.Subset(trainset, subset_indices)

# Incremento de batch de datos
batch_size = 128 if device.type == 'cuda' else 32
trainloader = torch.utils.data.DataLoader(subset, batch_size=batch_size, shuffle=True)

# Creamos la red
net = Net().to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)  # lr aumentado de 0.001 a 0.01

num_epochs = 10

print(f"\nEntrenando {num_epochs} épocas con modo RÁPIDO...")

import time
start_time = time.time()

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Época {epoch + 1}: pérdida = {running_loss/len(trainloader):.3f}')

elapsed_time = time.time() - start_time
print(f"\n✅ Completado en {elapsed_time:.1f} segundos!")


Entrenando 10 épocas con modo RÁPIDO...
Época 1: pérdida = 2.049
Época 2: pérdida = 1.570
Época 3: pérdida = 1.390
Época 4: pérdida = 1.269
Época 5: pérdida = 1.181
Época 6: pérdida = 1.112
Época 7: pérdida = 1.060
Época 8: pérdida = 1.010
Época 9: pérdida = 0.974
Época 10: pérdida = 0.924

✅ Completado en 128.9 segundos!


# Clasificamos y probamos si reconoce los sapos evaluandolo mediante la red

In [75]:
print("\n--- RESULTADOS ---")
net.eval()
for imagen in ['sapo1.jpg', 'sapo2.jpg', 'sapo3.jpg']:
    try:
        img = cv2.imread(imagen)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (32, 32))
        img_tensor = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = net(img_tensor)
            _, predicted = torch.max(output, 1)

        clase = classes[predicted[0]]

        if clase == 'frog':
            print(f"{imagen}: ✅ SÍ es sapo")
        else:
            print(f"{imagen}: ❌ NO es sapo (detectó: {clase})")
    except:
        print(f"{imagen}: Error al procesar")



--- RESULTADOS ---
sapo1.jpg: ✅ SÍ es sapo
sapo2.jpg: ✅ SÍ es sapo
sapo3.jpg: ✅ SÍ es sapo
